In [1]:
#Install dependencies
!pip install numpy
!pip install pandas 
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [2]:
#Import dependencies
import pandas as pd
import os, re
import numpy as np
import csv

#Paths
path_to_dataset = "/home/kuba/Desktop/ProjektBadawczoRozwojowy/test.parquet.gzip"

In [3]:
#Open dataset (parquet.gzip)
data = pd.read_parquet(path_to_dataset)
print(data.count()) 
print(data.lines)

datetime    10000
commit      10000
repo        10000
filepath    10000
content      9769
methods     10000
lines       10000
dtype: int64
1               [17, 24, 593]
4                        [62]
7         [3, 4, 5, 6, 7, 55]
8                       [449]
14       [30, 31, 33, 37, 39]
                 ...         
43486                      []
43494                      []
43524                      []
43538                      []
43562                      []
Name: lines, Length: 10000, dtype: object


In [61]:
#Save dataset as .csv file
new_dataset_csv = 'test.csv'
    
with open(new_dataset_csv, 'w', newline='') as file:
    writer = csv.writer(file)
    
    columns = ['File', 'Bug', 'SRC']
    writer.writerow(columns)
    
    i = 0
    files_with_bugs = 0
    all_files = 0
    good_files_with_bugs = 0
    
    for index, row in data.iterrows():
        
        all_files = all_files + 1
        if i != 0:
            csv_file = row.filepath
            line = row.lines

            if not np.size(line):
                csv_bug = False
            else:
                csv_bug = True
                good_files_with_bugs = good_files_with_bugs + 1

            try:
                csv_SRC = row.content.decode('UTF-8')
                writer.writerow([csv_file, csv_bug, csv_SRC])

            except:
                files_with_bugs = files_with_bugs + 1

        else:
            i = 1
            
    print("Files to trash: " + str(files_with_bugs))
    print("All files: " + str(all_files))
    print("Files to test: " + str(all_files - files_with_bugs))
    print("Files with bugs to test: " + str(good_files_with_bugs))
    print("Files without bugs to test: " + str(all_files - files_with_bugs - good_files_with_bugs))



Files to trash: 234
All files: 10000
Files to test: 9766
Files with bugs to test: 1245
Files without bugs to test: 8521


In [69]:
#It takes about ...
import pandas as pd
import os, re
import numpy as np

path_to_data_root_dir = '/home/kuba/Desktop/ProjektBadawczoRozwojowy/'
path_to_save_dir = "/home/kuba/Desktop/ProjektBadawczoRozwojowy/"

char_to_remove = ['+','-','*','/','=','++','--','\\','<str>','<char>','|','&','!']

def is_comment_line(code_line, comments_list):
    '''
        input
            code_line (string): source code in a line
            comments_list (list): a list that contains every comments
        output
            boolean value
    '''

    code_line = code_line.strip()

    if len(code_line) == 0:
        return False
    elif code_line.startswith('//'):
        return True
    elif code_line.startswith('#'):
        return True
    elif code_line.startswith('"""'):
        return True
    elif code_line in comments_list:
        return True
    
    return False

def is_empty_line(code_line):
    '''
        input
            code_line (string)
        output
            boolean value
    '''

    if len(code_line.strip()) == 0:
        return True

    return False

def preprocess_code_line(code_line):
    '''
        input
            code_line (string)
    '''

    code_line = re.sub("\'\'", "\'", code_line)
    code_line = re.sub("\".*?\"", "<str>", code_line)
    code_line = re.sub("\'.*?\'", "<char>", code_line)
    code_line = re.sub('\b\d+\b','',code_line)
    code_line = re.sub("\\[.*?\\]", '', code_line)
    code_line = re.sub("[\\.|,|:|;|{|}|(|)]", ' ', code_line)

    for char in char_to_remove:
        code_line = code_line.replace(char,' ')

    code_line = code_line.strip()

    return code_line

def create_code_df(code_str, filename):
    '''
        input
            code_str (string): a source code
            filename (string): a file name of source code

        output
            code_df (DataFrame): a dataframe of source code that contains the following columns
            - code_line (str): source code in a line
            - line_number (str): line number of source code line
            - is_comment (bool): boolean which indicates if a line is comment
            - is_blank_line(bool): boolean which indicates if a line is blank
    '''

    df = pd.DataFrame()

    code_lines = code_str.splitlines()
    
    preprocess_code_lines = []
    is_comments = []
    is_blank_line = []


    comments = re.findall(r'("""[\s\S]*?""")|\'\'\'[\s\S]*?\'\'\'',code_str,re.DOTALL)
    comments_str = '\n'.join(comments)
    comments_list = comments_str.split('\n')
    
    for index, comment in enumerate(comments_list):
        comments_list[index] = comment.lstrip()

    for l in code_lines:
        l = l.strip()
        is_comment = is_comment_line(l,comments_list)
        is_comments.append(is_comment)
        # preprocess code here then check empty line...

        if not is_comment:
            l = preprocess_code_line(l)
            
        is_blank_line.append(is_empty_line(l))
        preprocess_code_lines.append(l)

    if 'test' in filename:
        is_test = True
    else:
        is_test = False

    df['filename'] = [filename]*len(code_lines)
    df['is_test_file'] = [is_test]*len(code_lines)
    df['code_line'] = preprocess_code_lines
    df['line_number'] = np.arange(1,len(code_lines)+1)
    df['is_comment'] = is_comments
    df['is_blank'] = is_blank_line

    return df

def preprocess_data():
    
    file_level_data = pd.read_csv(path_to_data_root_dir + 'test.csv', encoding='latin')
    file_level_data = file_level_data.fillna('')

    preprocessed_df_list = []

    for idx, row in file_level_data.iterrows():

        filename = row['File']

        if '.py' not in filename:
            continue

        code = row['SRC']
        label = row['Bug']

        code_df = create_code_df(code, filename)
        code_df['file-label'] = [False]*len(code_df)
        code_df['line-label'] = [False]*len(code_df)
                
        if len(code_df) > 0:
            preprocessed_df_list.append(code_df)

        all_df = pd.concat(preprocessed_df_list)
        all_df.to_csv(path_to_save_dir + "result.csv",index=False)
        print(str(idx) + "...")

        
preprocess_data()
print("DONE")


0...
1...
2...
3...
4...
5...
6...
7...
8...
9...
10...
11...
12...
13...
14...
15...
16...
17...
18...
19...
20...
21...
22...
23...
24...
25...
26...
27...
28...
29...
30...
31...
32...
33...
34...
35...
36...
37...
38...
39...
40...
41...
42...
43...
44...
45...
46...
47...
48...
49...
50...
51...
52...
53...
54...
55...
56...
57...
58...
59...
60...
61...
62...
63...
64...
65...
66...
67...
68...
69...
70...
71...
72...
73...
74...
75...
76...
77...
78...
79...
80...
81...
82...
83...
84...
85...
86...


KeyboardInterrupt: 

In [38]:
# import pandas as pd
# import os, re
# import numpy as np

# # from my_util import *

# data_root_dir = '../datasets/original/'
# save_dir = "../datasets/preprocessed_data/"

# char_to_remove = ['+','-','*','/','=','++','--','\\','<str>','<char>','|','&','!']

# if not os.path.exists(save_dir):
#     os.makedirs(save_dir)

# file_lvl_dir = data_root_dir+'File-level/'
# line_lvl_dir = data_root_dir+'Line-level/'


# def is_comment_line(code_line, comments_list):
#     '''
#         input
#             code_line (string): source code in a line
#             comments_list (list): a list that contains every comments
#         output
#             boolean value
#     '''

#     code_line = code_line.strip()

#     if len(code_line) == 0:
#         return False
#     elif code_line.startswith('//'):
#         return True
#     elif code_line in comments_list:
#         return True
    
#     return False

# def is_empty_line(code_line):
#     '''
#         input
#             code_line (string)
#         output
#             boolean value
#     '''

#     if len(code_line.strip()) == 0:
#         return True

#     return False

# def preprocess_code_line(code_line):
#     '''
#         input
#             code_line (string)
#     '''

#     code_line = re.sub("\'\'", "\'", code_line)
#     code_line = re.sub("\".*?\"", "<str>", code_line)
#     code_line = re.sub("\'.*?\'", "<char>", code_line)
#     code_line = re.sub('\b\d+\b','',code_line)
#     code_line = re.sub("\\[.*?\\]", '', code_line)
#     code_line = re.sub("[\\.|,|:|;|{|}|(|)]", ' ', code_line)

#     for char in char_to_remove:
#         code_line = code_line.replace(char,' ')

#     code_line = code_line.strip()

#     return code_line

# def create_code_df(code_str, filename):
#     '''
#         input
#             code_str (string): a source code
#             filename (string): a file name of source code

#         output
#             code_df (DataFrame): a dataframe of source code that contains the following columns
#             - code_line (str): source code in a line
#             - line_number (str): line number of source code line
#             - is_comment (bool): boolean which indicates if a line is comment
#             - is_blank_line(bool): boolean which indicates if a line is blank
#     '''

#     df = pd.DataFrame()

#     code_lines = code_str.splitlines()
    
#     preprocess_code_lines = []
#     is_comments = []
#     is_blank_line = []


#     comments = re.findall(r'(/\*[\s\S]*?\*/)',code_str,re.DOTALL)
#     comments_str = '\n'.join(comments)
#     comments_list = comments_str.split('\n')
#     print(comments_list)

#     for l in code_lines:
#         l = l.strip()
#         is_comment = is_comment_line(l,comments_list)
#         is_comments.append(is_comment)
#         # preprocess code here then check empty line...

#         if not is_comment:
#             l = preprocess_code_line(l)
            
#         is_blank_line.append(is_empty_line(l))
#         preprocess_code_lines.append(l)

#     if 'test' in filename:
#         is_test = True
#     else:
#         is_test = False

#     df['filename'] = [filename]*len(code_lines)
#     df['is_test_file'] = [is_test]*len(code_lines)
#     df['code_line'] = preprocess_code_lines
#     df['line_number'] = np.arange(1,len(code_lines)+1)
#     df['is_comment'] = is_comments
#     df['is_blank'] = is_blank_line

#     return df

# def preprocess_data():

#     #cur_all_rel = all_releases[proj_name]

#     #for rel in cur_all_rel:
#     file_level_data = pd.read_csv('/home/kuba/Desktop/DeepLine/DeepLineDP_with_line-level_baseline_implementation/datasets/original/File-level/activemq-5.0.0_ground-truth-files_dataset.csv', encoding='latin')
#     #line_level_data = pd.read_csv(line_lvl_dir+rel+'_defective_lines_dataset.csv', encoding='latin')

#     file_level_data = file_level_data.fillna('')

#     #buggy_files = list(line_level_data['File'].unique())

#     preprocessed_df_list = []

#     for idx, row in file_level_data.iterrows():

#         filename = row['File']

#         if '.java' not in filename:
#             continue

#         code = row['SRC']
#         label = row['Bug']

#         code_df = create_code_df(code, filename)
#         code_df['file-label'] = [label]*len(code_df)
#         code_df['line-label'] = [False]*len(code_df)

# #         if filename in buggy_files:
# #             buggy_lines = list(line_level_data[line_level_data['File']==filename]['Line_number'])
# #             code_df['line-label'] = code_df['line_number'].isin(buggy_lines)

#         if len(code_df) > 0:
#             preprocessed_df_list.append(code_df)

#     all_df = pd.concat(preprocessed_df_list)
#     all_df.to_csv("testt.csv",index=False)
#     print('finish release {}')

# #for proj in list(all_releases.keys()):
# preprocess_data()


['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the NOTICE file distributed with', '* this work for additional information regarding copyright ownership.', '* The ASF licenses this file to You under the Apache License, Version 2.0', '* (the "License"); you may not use this file except in compliance with', '* the License.  You may obtain a copy of the License at', '*', '*      http://www.apache.org/licenses/LICENSE-2.0', '*', '* Unless required by applicable law or agreed to in writing, software', '* distributed under the License is distributed on an "AS IS" BASIS,', '* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.', '* See the License for the specific language governing permissions and', '* limitations under the License.', '*/', '/**', '* Establishes a connection to the remote broker specified by the broker', '* url.', '*', '* @return - connection to the broker', '* @throws JMSException',

['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the NOTICE file distributed with', '* this work for additional information regarding copyright ownership.', '* The ASF licenses this file to You under the Apache License, Version 2.0', '* (the "License"); you may not use this file except in compliance with', '* the License.  You may obtain a copy of the License at', '*', '*      http://www.apache.org/licenses/LICENSE-2.0', '*', '* Unless required by applicable law or agreed to in writing, software', '* distributed under the License is distributed on an "AS IS" BASIS,', '* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.', '* See the License for the specific language governing permissions and', '* limitations under the License.', '*/', '/**', '* A utility class used by the Session for dispatching messages asynchronously', '* to consumers', '*', '* @version $Revision$', '* @see javax.jms.Session'

['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the NOTICE file distributed with', '* this work for additional information regarding copyright ownership.', '* The ASF licenses this file to You under the Apache License, Version 2.0', '* (the "License"); you may not use this file except in compliance with', '* the License.  You may obtain a copy of the License at', '*', '*      http://www.apache.org/licenses/LICENSE-2.0', '*', '* Unless required by applicable law or agreed to in writing, software', '* distributed under the License is distributed on an "AS IS" BASIS,', '* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.', '* See the License for the specific language governing permissions and', '* limitations under the License.', '*/', '/**', '* Resets the statistics', '*/', '/**', '* enable statistics gathering', '*/', '/**', '* disable statistics gathering', '*/', '/**', '* Returns true if sta

['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the NOTICE file distributed with', '* this work for additional information regarding copyright ownership.', '* The ASF licenses this file to You under the Apache License, Version 2.0', '* (the "License"); you may not use this file except in compliance with', '* the License.  You may obtain a copy of the License at', '*', '*      http://www.apache.org/licenses/LICENSE-2.0', '*', '* Unless required by applicable law or agreed to in writing, software', '* distributed under the License is distributed on an "AS IS" BASIS,', '* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.', '* See the License for the specific language governing permissions and', '* limitations under the License.', '*/', '/**', '* Abstraction to allow different recovery policies to be plugged', '* into the region implementations.  This is used by a topic to retroactively recover',

['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the NOTICE file distributed with', '* this work for additional information regarding copyright ownership.', '* The ASF licenses this file to You under the Apache License, Version 2.0', '* (the "License"); you may not use this file except in compliance with', '* the License.  You may obtain a copy of the License at', '*', '*      http://www.apache.org/licenses/LICENSE-2.0', '*', '* Unless required by applicable law or agreed to in writing, software', '* distributed under the License is distributed on an "AS IS" BASIS,', '* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.', '* See the License for the specific language governing permissions and', '* limitations under the License.', '*/', '/**', '* @openwire:marshaller', '* @version $Revision: 1.10 $', '*/', '/**', '* @openwire:property version=1', '*/']
['/**', '* Licensed to the Apache Software F

...

['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the NOTICE file distributed with', '* this work for additional information regarding copyright ownership.', '* The ASF licenses this file to You under the Apache License, Version 2.0', '* (the "License"); you may not use this file except in compliance with', '* the License.  You may obtain a copy of the License at', '*', '*      http://www.apache.org/licenses/LICENSE-2.0', '*', '* Unless required by applicable law or agreed to in writing, software', '* distributed under the License is distributed on an "AS IS" BASIS,', '* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.', '* See the License for the specific language governing permissions and', '* limitations under the License.', '*/', '/**', '* @version $Revision$', '*/']
['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the 

['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the NOTICE file distributed with', '* this work for additional information regarding copyright ownership.', '* The ASF licenses this file to You under the Apache License, Version 2.0', '* (the "License"); you may not use this file except in compliance with', '* the License.  You may obtain a copy of the License at', '*', '*      http://www.apache.org/licenses/LICENSE-2.0', '*', '* Unless required by applicable law or agreed to in writing, software', '* distributed under the License is distributed on an "AS IS" BASIS,', '* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.', '* See the License for the specific language governing permissions and', '* limitations under the License.', '*/', '/**', '* Test case for the OpenWire marshalling for ConnectionInfo NOTE!: This file is', '* auto generated - do not modify! if you need to make a change, please 

['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the NOTICE file distributed with', '* this work for additional information regarding copyright ownership.', '* The ASF licenses this file to You under the Apache License, Version 2.0', '* (the "License"); you may not use this file except in compliance with', '* the License.  You may obtain a copy of the License at', '*', '*      http://www.apache.org/licenses/LICENSE-2.0', '*', '* Unless required by applicable law or agreed to in writing, software', '* distributed under the License is distributed on an "AS IS" BASIS,', '* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.', '* See the License for the specific language governing permissions and', '* limitations under the License.', '*/', '/**', '* Useful base class for unit test cases', '*', '* @version $Revision: 1.4 $', '*/', '/**', '* Creates an ActiveMQMessage.', '*', '* @return ActiveMQMessag

['/**', '* Licensed to the Apache Software Foundation (ASF) under one or more', '* contributor license agreements.  See the NOTICE file distributed with', '* this work for additional information regarding copyright ownership.', '* The ASF licenses this file to You under the Apache License, Version 2.0', '* (the "License"); you may not use this file except in compliance with', '* the License.  You may obtain a copy of the License at', '*', '*      http://www.apache.org/licenses/LICENSE-2.0', '*', '* Unless required by applicable law or agreed to in writing, software', '* distributed under the License is distributed on an "AS IS" BASIS,', '* WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.', '* See the License for the specific language governing permissions and', '* limitations under the License.', '*/', '/**', '* @version $Revisio n$ TODO: Must override equals and hashCode (JCA spec 16.4)', '* @org.apache.xbean.XBean element="managedConnectionFactory"', '*/', '/*

finish release {}
